In [1]:
import plotly.express as px
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

#import csv
pd.set_option('display.max_rows', None)  # Display all rows


In [2]:
if not os.path.exists("images"):
    os.mkdir("images")

day = "28"
month = "09"
dataset = "rfid"

date = day + "_" + month #change date accordingly
date_year = "2023-" + month + "-" + day

folder_path = dataset + '_' + date + '/'


def process_file(file_number, date):
    file_path = os.path.join(folder_path, f'rfid_{date}_{file_number}.csv')

    if file_number == "final":
        df = pd.read_csv(file_path, parse_dates=['Stamp'])
        if df.shape[1] == 4:
            df = df.drop(df.columns[3], axis=1)
            df = df.drop(df.columns[1], axis=1)

    else:
        df = pd.read_csv(file_path, parse_dates=['ts'])

        # Remove first empty column if it exists
        if df.shape[1] == 3:
            df = df.drop(df.columns[0], axis=1)
        # Remove entries with 'gabarit' equal to 0
        df= df[df['gabarit'] != 0]

        # Write the updated DataFrame back to the CSV file
        df.to_csv(file_path, index = False)
        # Reset the index of the DataFrame
        df = df.reset_index(drop = True)

    return df

df1 = process_file("1",date)
df2 = process_file("2",date)
df3 = process_file("3",date)
df4 = process_file("final",date)



In [31]:

def time_in_station_fun(df):
    """
    Calculates and records the time spent in a station for each occurrence of a 'gabarit' in a DataFrame in seconds.

    Parameters:
        - df: DataFrame containing timestamps and 'gabarit' values.

    Returns:
        DataFrame with an added 'time_in_station' column representing the time
        spent in the station for each occurrence (in seconds).
    """
    
    curr_gabarit = 0
    index_i = 0  # Initialize index_i outside the loop
    index_f = 0  # Initialize index_f outside the loop
    cnt = 0
    df_in = pd.DataFrame(columns=['ts', 'gabarit', 'time_in_station'])
    #df_in['ts'] = pd.to_datetime(df['ts'])

    for i in range(df.shape[0]): #0 to 129
        if i != 0: # not first index
            if ((df.loc[i, 'gabarit'] != curr_gabarit)):
                # routine to present final result
                df_in, cnt = calc_time_in_station(df, df_in, index_i, index_f, cnt)
            
          
            # new gabarit is found, i = f
            if df.loc[i, 'gabarit'] != df.loc[i-1, 'gabarit']:
                curr_gabarit = df.loc[i, 'gabarit']
                index_i = i # starts the interval
                index_f = i
        else: # for first entry, i = f
            curr_gabarit = df.loc[i, 'gabarit']
            index_i = i
            index_f = i

        
        if i != df.shape[0] - 1: # not last index 
            if df.loc[i, 'gabarit'] == df.loc[i+1, 'gabarit']:
                index_f = i+1
        else:
            index_f = i           
            df_in, cnt = calc_time_in_station(df, df_in, index_i, index_f, cnt)

    return df_in


# calculates time and writes it in dataframe
def calc_time_in_station(df, df_in, index_i, index_f, cnt):
    if index_f - index_i == 0:
        time_in_station = 1
    else: 
        time_in_station = ((df.loc[index_f, 'ts'] - df.loc[index_i, 'ts']).total_seconds() + 1)

    #print(f"time_in_station is {time_in_station}")
    df_in.loc[cnt,'ts'] = df.loc[index_i, 'ts']
    df_in.loc[cnt,'gabarit'] = df.loc[index_i, 'gabarit']
    df_in.loc[cnt,'time_in_station'] = time_in_station

    #df.loc[index_f, 'time_in_station'] = round(time_in_station,2)
    cnt += 1

    return df_in, cnt


def plot_time_in_station(df, station):
    df = df[df['time_in_station'] != 0]
    # Create a scatter plot using Plotly Express
    if station == 1:
        title_plot_var = 'Begginning of Stage 3'
    elif station == 2:
        title_plot_var = 'End of Stage 3'
    elif station == 3:
        title_plot_var = 'Begginning of Stage 7'

    title_plot = 'Plot of Time in ' + title_plot_var + ' over Time'

    ts_values = np.array(df['ts'])
    df['time_in_station'] = df['time_in_station'] / 60

    # Create a scatter plot using go.Scatter
    scatter = go.Scatter(x=ts_values, y=df['time_in_station'], mode='markers', 
                        name='Scatter Plot', marker=dict(color='blue'))

    # Create a layout
    layout = go.Layout(
        title=title_plot,
        xaxis=dict(title='Timestamp', range=[date_year + ' 09:00:00', date_year + ' 18:00:00']),  # Set the range for the x-axis
        yaxis=dict(title='Time in Station', range=[-0.2, 3])  # Set the range for the y-axis
    )

    # Create a figure and add the scatter trace
    fig = go.Figure(data=[scatter], layout=layout)

    # Show the plot
    fig.show()
    fig.write_image("images/fig_" + dataset + "_" + day + "_" + month + "_" + str(station) + ".svg")

df_in_2 = time_in_station_fun(df2)

In [16]:
def remove_outliers(df, max_time):
    # Ensure the column 'time_in_station' exists
    if 'time_in_station' not in df.columns:
        print("Error: 'time_in_station' column not found.")
        return df
    
    

    # Remove rows where 'time_in_station' is greater than 5 minutes
    df = df[df['time_in_station'] <= max_time / 60]

    return df

In [69]:
plot = 1
df_in_1 = time_in_station_fun(df1)
#df_in_1 = remove_outliers(df_in_1, 5)
display(df_in_1)


df_in_2 = time_in_station_fun(df2)
#df_in_2 = remove_outliers(df_in_2, 5)
display(df_in_2)

df_in_3 = time_in_station_fun(df3)
#df_in_3 = remove_outliers(df_in_3, 5)
display(df_in_3)

if plot:
    plot_time_in_station(df_in_1,1)
    plot_time_in_station(df_in_2,2)
    plot_time_in_station(df_in_3,3)

,ts,gabarit,time_in_station
0,2023-09-28 09:10:26,14,1
1,2023-09-28 09:13:02,23,1
2,2023-09-28 09:13:52,17,1
3,2023-09-28 09:16:14,21,1
4,2023-09-28 09:17:43,6,10.0
5,2023-09-28 09:20:33,19,23.0
6,2023-09-28 09:23:32,4,1
7,2023-09-28 09:25:44,9,1
8,2023-09-28 09:27:16,3,26.0
9,2023-09-28 09:28:57,8,1


,ts,gabarit,time_in_station
0,2023-09-28 09:07:50,11,1
1,2023-09-28 09:11:48,12,60.0
2,2023-09-28 09:13:54,17,1
3,2023-09-28 09:15:22,13,1
4,2023-09-28 09:27:20,3,1
5,2023-09-28 09:28:58,8,1
6,2023-09-28 09:31:35,15,1
7,2023-09-28 09:34:23,13,1
8,2023-09-28 09:36:58,19,1
9,2023-09-28 09:38:39,24,1


,ts,gabarit,time_in_station
0,2023-09-28 09:07:33,24,19.0
1,2023-09-28 09:08:04,19,36.0
2,2023-09-28 09:12:38,22,53.0
3,2023-09-28 09:16:29,12,1
4,2023-09-28 09:18:20,23,1
5,2023-09-28 09:20:14,17,9.0
6,2023-09-28 09:31:18,9,1
7,2023-09-28 09:32:00,3,8.0
8,2023-09-28 09:32:59,8,1
9,2023-09-28 09:35:53,15,13.0


In [7]:
def test(df1):
    df1 = df1.iloc[0:10]
    return df1

df1 = test(df1)
display(df1)

,ts,gabarit,time_in_station
0,2023-09-28 09:10:26,14,0.02
1,2023-09-28 09:13:02,23,0.02
2,2023-09-28 09:13:52,17,0.02
3,2023-09-28 09:16:14,21,0.02
4,2023-09-28 09:17:43,6,0.00
5,2023-09-28 09:17:52,6,0.17
6,2023-09-28 09:20:33,19,0.00
7,2023-09-28 09:20:55,19,0.38
8,2023-09-28 09:23:32,4,0.02
9,2023-09-28 09:25:44,9,0.02


In [81]:


def time_between_station(df1, df2, station1, station2, max_time):
    """
    Finds and records the time difference between occurrences of the same 'gabarit' in two DataFrames,
    df1 and df2, within a specified time window.

    Parameters:
        - df1: DataFrame containing timestamps and 'gabarit' values for station1.
        - df2: DataFrame containing timestamps and 'gabarit' values for station2.
        - station1: Identifier for the first station.
        - station2: Identifier for the second station.
        - max_time: Maximum time window to search for matching occurrences (in minutes).
    
    Returns:
        DataFrame with columns 'gabarit', 'ts' (timestamp), and 'time_bet_station1_station2'
        representing the time difference (in minutes) between corresponding occurrences in
        station1 and station2 within the specified time window.
    """
    
    df_bet = pd.DataFrame(columns=['gabarit', f'time_bet_{station1}_{station2}'])
    df_bet['ts'] = pd.NaT  # initialize as datetime type

    print(station1 != 3 and station2 != 4)
    print(station1 != 3 & station2 != 4)
    print(station1 != 3 & station2 != 4)
    if (station1 != 3 & station2 != 4): ##1 2 or 2 3 
        column_order = ['ts', 'gabarit', f'time_bet_{station1}_{station2}']
        df_bet = df_bet[column_order]
        print(f'1: {station1}') 
        print(f'2: {station2}')
        df_bet = find_matching_gabarit(df1, df2, station1, station2, max_time, df_bet)
    else:
        df_bet['model'] = np.nan
        column_order = ['ts', 'gabarit', 'model', f'time_bet_{station1}_{station2}']
        df_bet = df_bet[column_order]
        df_bet = find_matching_gabarit_final(df1, df2, df_bet)

    return df_bet


def find_matching_gabarit(df1,df2,station1, station2, max_time, df_bet):

    cnt = 0
            
    for index in range(df1.shape[0]):
        start_time = df1.loc[index, 'ts']
        end_time = start_time + pd.Timedelta(minutes=max_time)
        matching_rows = df2[(df2['ts'] >= start_time) & 
                            (df2['ts'] <= end_time) 
                                & (df2['gabarit'] == df1.loc[index,'gabarit'])]
        if not matching_rows.empty:
            first_occurrence = matching_rows.iloc[0]
            
            # df1: initial time stamp, time_in_station_1
            # df2: initial time stamp, time_in_station_2
            if (station1 == 1 & station2 == 2):
                time_bet = (first_occurrence['time_in_station'] +
                        (first_occurrence['ts'] - df1.loc[index, 'ts']).total_seconds()) # in seconds
            else:
                time_bet = ((first_occurrence['ts'] - df1.loc[index, 'ts'] # in seconds
                            - pd.Timedelta(seconds = df1.loc[index, 'time_in_station'])).total_seconds())

            #if (first_occurrence['ts'] - df1.loc[index, 'ts']).total_seconds() < df1.loc[index, 'time_in_station']:
                #df1: 10:06:00 17
                #     10:07:30 17
                #df2: 10:07:10 17
            #    time_bet = (df1.loc[index, 'time_in_station'] + first_occurrence['time_in_station'])

            df_bet.loc[cnt, 'ts'] = df1.loc[index, 'ts']  # Copy the timestamp for reference
            df_bet.loc[cnt, 'gabarit'] = first_occurrence['gabarit']
            df_bet.loc[cnt, f'time_bet_{station1}_{station2}'] = time_bet #round,2
            cnt += 1
        
    return df_bet

def find_matching_gabarit_final(df1, df2, df_bet):

    cnt = 0
            
    for index in range(df1.shape[0]):
        start_time = (df1.loc[index, 'ts'] + pd.Timedelta(seconds = df1.loc[index, 'time_in_station'])
                      + pd.Timedelta(seconds=20)) # not possible to have a test that takes less than 20 seconds
        matching_rows = df2[(df2['Stamp'] >= start_time)]


        if not matching_rows.empty:
            first_occurrence = matching_rows.iloc[0]
            
            # df1: initial time stamp, time_in_station_1
            # df2: initial time stamp, time_in_station_2
            time_bet = ((first_occurrence['Stamp'] - df1.loc[index, 'ts']).total_seconds()) # in minutes

            df_bet.loc[cnt, 'ts'] = df1.loc[index, 'ts']  # Copy the timestamp for reference
            df_bet.loc[cnt, 'gabarit'] = df1.loc[index,'gabarit']
            #print(cnt)
            #print(type(first_occurrence['Model']))
            #print(first_occurrence['Model'])
            df_bet.loc[cnt, 'model'] = str(first_occurrence['Model'])
            df_bet.loc[cnt, f'time_bet_3_4'] = time_bet
            cnt += 1
        
    return df_bet


def all_time_between_station(df12, df23, df34):
    #df_bet_all = pd.DataFrame(columns = ['ts1','time_bet_12','ts2', 'ts2','time_bet_23','ts3','time_bet_34','gabarit','model'])
    df_bet_all = pd.DataFrame(columns = ['ts1','gabarit', 'model','time_bet_1_2','time_bet_2_3','time_bet_3_4']) 
    
    station1 = 1
    station2 = 2
    station3 = 3
    station4 = 4
    cnt = 0
    cnt2 = 0
    for index in range(df12.shape[0]):
        start_time = (df12.loc[index, 'ts'] + pd.Timedelta(seconds = df12.loc[index, f'time_bet_{station1}_{station2}'])
                       - pd.Timedelta(seconds=2))
        end_time = start_time + pd.Timedelta(seconds=3)

        matching_rows = df23[(df23['ts'] >= start_time) & (df23['ts'] <= end_time) 
                                & (df23['gabarit'] == df12.loc[index,'gabarit'])]
        if not matching_rows.empty:
            first_occurrence = matching_rows.iloc[0]
            #total_time = (first_occurrence['time_in_station'] - df1.loc[index, 'ts']).total_seconds()) # in seconds
            print(df12.loc[index] )
            print(first_occurrence)

            #df_bet_all.loc[cnt, 'ts1'] = df12.loc[index, 'ts']  # Copy the timestamp for reference
            #df_bet_all.loc[cnt, 'gabarit'] = first_occurrence['gabarit']
            #df_bet_all.loc[cnt, f'time_bet_{station1}_{station2}'] = df12.loc[index, f'time_bet_{station1}_{station2}'] #round,2
            cnt += 1

            start_time2 = (first_occurrence['ts'] + pd.Timedelta(seconds = first_occurrence[f'time_bet_{station2}_{station3}']) 
                          - pd.Timedelta(seconds=2))
            end_time2 = start_time2 +   pd.Timedelta(seconds=3)
            print(first_occurrence['ts'])
            print(first_occurrence[f'time_bet_{station2}_{station3}'])

            print(start_time2)
            print(end_time2)
            matching_rows2 = df34[(df34['ts'] >= start_time2) & (df34['ts'] <= end_time2) 
                                & (df34['gabarit'] == first_occurrence['gabarit'])]
            
            if not matching_rows2.empty:
                first_occurrence2 = matching_rows2.iloc[0]
                #total_time = (first_occurrence['time_in_station'] - df1.loc[index, 'ts']).total_seconds()) # in seconds

                print(first_occurrence2)
                print()
           
                #df_bet_all.loc[cnt2, f'time_bet_{station3}_{station4}'] = first_occurrence2[f'time_bet_{station3}_{station4}']
                #df_bet_all.loc[cnt2, 'model'] = first_occurrence2['Model']
                
                #print(first_occurrence['ts'])
                cnt2 += 1
                



    return df_bet_all

df_bet_12 = time_between_station(df_in_1, df_in_2, 1, 2, 5)
#df_bet_12_ = time_between_station(df1, df2, 1, 2, 4)

display(df_bet_12)
#display(df_bet_12_)

df_bet_23 = time_between_station(df_in_2, df_in_3, 2, 3, 20)
#df_bet_23_ = time_between_station(df2, df3, 2, 3, 200,0)

display(df_bet_23)
#display(df_bet_23_)

df_bet_34 = time_between_station(df_in_3, df4, 3, 4, 0)

display(df_bet_34)
#display(df_bet_34_)

df_bet_all = all_time_between_station(df_bet_12, df_bet_23,df_bet_34)
display(df_bet_all)


    


SyntaxError: invalid syntax (433653328.py, line 24)

In [ ]:
def plot_time_bet_stations(df_in1, df_in2, df_in3, df_bet12, df_bet23, df_bet34 ):

    df_bet = pd.DataFrame(columns=['ts','gabarit', 'model', 'time_bet_1_2', 'time_bet_2_3', 'time_bet_3_4'])
    df_bet['ts'] = pd.NaT  # initialize as datetime type
    column_order = ['ts','gabarit', 'model', 'time_bet_1_2', 'time_bet_2_3', 'time_bet_3_4']
    df_bet = df_bet[column_order]


    df1['ts'] - que nao e 0 onde comecamos 
    somamos este time_in_station a esse ts 
    somamos time between station 1 e 2 
    e somamos time in station que nao seja 0

    df1[(df1['time_in_station' != 0])] sacar gabarit e ts
    

    non_zero_indexes = df1[df1['time_in_station'] != 0].index
    matches = []
    cnt = 0
    
    for index in non_zero_indexes:
        start_time = df1.loc[index, 'ts']
        end_time = start_time + pd.Timedelta(minutes=max_time)
        matching_rows = df2[(df2['ts'] >= start_time) & (df2['ts'] <= end_time) 
                            & (df2['gabarit'] == df1.loc[index,'gabarit'])]
        
        if not matching_rows.empty:
            # Find the first occurrence
            first_occurrence = matching_rows.iloc[0]
            matches.append((index, first_occurrence.name))
            delta_t = (first_occurrence['ts'] - df1.loc[index,'ts']).total_seconds()
            df_bet.loc[cnt, 'ts'] = df1.loc[index, 'ts']  # Copy the timestamp for reference
            df_bet.loc[cnt, 'gabarit'] = first_occurrence['gabarit']
            df_bet.loc[cnt, f'time_bet_{station1}_{station2}'] = round((delta_t / 60),2)
            cnt += 1

    return df_bet

def find_matching_gabarit_final(df1, df2, df_bet):

    non_zero_indexes = df1[df1['time_in_station'] != 0].index
    matches = []
    cnt = 0
    
    for index in non_zero_indexes:
        start_time = df1.loc[index, 'ts'] + pd.Timedelta(seconds=20) # not possible to have a test that takes less than 20 seconds
        #end_time = start_time + pd.Timedelta(minutes=max_time)
        matching_rows = df2[(df2['Stamp'] >= start_time)]
        
        if not matching_rows.empty:
            # Find the first occurrence
            first_occurrence = matching_rows.iloc[0]
            matches.append((index, first_occurrence.name))
            delta_t = (first_occurrence['Stamp'] - df1.loc[index,'ts']).total_seconds()
            df_bet.loc[cnt, 'ts'] = df1.loc[index, 'ts']  # Copy the timestamp for reference
            df_bet.loc[cnt, 'gabarit'] = df1.loc[index,'gabarit']
            #print(df2[first_occurrence['Model']])
            #df_bet.loc[cnt, 'model'] = "SPT120018W"
            df_bet.loc[cnt, 'model'] = first_occurrence['Model']
            df_bet.loc[cnt, f'time_bet_3_4'] = round((delta_t ),2)
            cnt += 1

    return